
# Chapter 5 — Interactive Tools (with sliders)
Biofabrication Syllabus (2025) — **Bioreactors and Tissue Maturation**

This notebook provides **interactive** versions of the quantitative tools using `ipywidgets`:
1. **Fick's Law Visualizer (1D)** — explore diffusion with optional first‑order consumption.
2. **Shear Stress Calculator** — estimate wall shear stress in a perfused rectangular channel.
3. **Perfusion Design Helper** — compute Péclet and Damköhler numbers and interpret regimes.

> Plots use only `matplotlib` and each has its own figure.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from ipywidgets import interact, interactive, fixed, HBox, VBox, FloatSlider, IntSlider, Dropdown, Checkbox, Layout
from IPython.display import display, Markdown



## 1) Fick's Law Visualizer — 1D diffusion with optional consumption

Equation:  
\[\frac{\partial C}{\partial t} = D \frac{\partial^2 C}{\partial x^2} - k C\]
- Domain: \(x \in [0, L]\) (slab geometry)
- BCs: Dirichlet at \(x=0\) (fixed \(C_0\)), zero‑flux at \(x=L\)
- IC: \(C(x,0)=0\)

**Tips:**  
- Set `k = 0` to observe pure diffusion.  
- Reduce `D` or increase `k` to see steeper gradients and smaller penetration depth.  
- Use `dx` and `max time` to control resolution and runtime.


In [ ]:

def simulate_diffusion(D=1e-10, k=5e-6, L=1e-3, C0=1.0, dx=1e-5, t_max=7200, n_curves=5):
    # Build grid
    nx = int(L/dx) + 1
    x  = np.linspace(0, L, nx)
    # stable dt for explicit scheme
    dt_stable = dx**2/(2*D) if D > 0 else 1.0
    dt = min(0.4*dt_stable, 0.25)  # cap dt to keep compute time reasonable
    nsteps = int(np.ceil(t_max/dt))
    # times to evaluate
    t_eval = np.linspace(t_max/n_curves, t_max, n_curves)
    # init
    C = np.zeros_like(x)
    profiles = {}
    t = 0.0
    for step in range(nsteps+1):
        # store at nearest steps
        for te in t_eval:
            if abs(t - te) <= 0.5*dt and te not in profiles:
                profiles[te] = C.copy()
        if t >= t_max:
            break
        # second derivative
        d2Cdx2 = np.zeros_like(C)
        d2Cdx2[1:-1] = (C[2:] - 2*C[1:-1] + C[:-2]) / dx**2
        # BCs
        C[0] = C0                       # Dirichlet
        d2Cdx2[-1] = (C[-2] - C[-1]) / dx**2  # approx zero-flux
        # update
        C = C + dt*(D*d2Cdx2 - k*C)
        t += dt

    # Plot
    plt.figure()
    for te in sorted(profiles.keys()):
        plt.plot(x*1e3, profiles[te], label=f"t ≈ {int(te)} s")
    plt.xlabel("Depth x (mm)")
    plt.ylabel("Concentration C (a.u.)")
    plt.title("Diffusion–reaction profiles in a 1D slab")
    plt.legend()
    plt.show()

    # Penetration depth estimate
    if k > 0:
        Lp = sqrt(D/k)
        display(Markdown(f"**Estimated penetration depth**: \(L_p \\approx {Lp*1e3:.2f}\\) mm for given D and k."))
    else:
        display(Markdown("**No consumption (k=0)**: penetration depth grows with \(\\sqrt{Dt}\\)."))

# Interactive controls
_ = interact(
    simulate_diffusion,
    D=FloatSlider(value=1e-10, min=1e-12, max=5e-10, step=1e-12, readout_format='.1e', description='D (m²/s)'),
    k=FloatSlider(value=5e-6,  min=0.0,   max=5e-5,  step=1e-7,  readout_format='.1e', description='k (1/s)'),
    L=FloatSlider(value=1e-3,  min=2e-4,  max=5e-3,  step=1e-4,  readout_format='.1e', description='L (m)'),
    C0=FloatSlider(value=1.0,  min=0.1,   max=2.0,   step=0.1,   description='C₀ (a.u.)'),
    dx=FloatSlider(value=1e-5, min=5e-6,  max=5e-5,  step=5e-6,  readout_format='.1e', description='dx (m)'),
    t_max=IntSlider(value=7200, min=600, max=43200, step=600, description='max time (s)'),
    n_curves=IntSlider(value=5, min=3, max=8, step=1, description='# curves')
)



## 2) Shear Stress Calculator — rectangular perfusion channel

Approximation for wide channels \((b \gg h)\):  
\[\tau \approx \frac{6 \mu Q}{b h^2}\]

- \(\mu\): dynamic viscosity (Pa·s) — water-like media ≈ \(1\times10^{-3}\) Pa·s at 37 °C  
- \(Q\): volumetric flow (m³/s) — 1 mL/min = \(1\times10^{-6}/60\) m³/s  
- \(b\): width (m)  
- \(h\): height (m)

**Tip:** Compare with physiological ranges (e.g., endothelial cells: 0.5–2 Pa).


In [ ]:

def shear_stress(mu=1e-3, Q_mLmin=1.0, b=1e-2, h=5e-4):
    Q = Q_mLmin*1e-6/60.0  # convert to m^3/s
    tau = 6.0*mu*Q/(b*h**2)
    plt.figure()
    # small sweep around chosen Q to visualize sensitivity
    Qs = np.linspace(0.1*Q_mLmin, 2.0*Q_mLmin, 20)
    taus = 6.0*mu*(Qs*1e-6/60.0)/(b*h**2)
    plt.plot(Qs, taus)
    plt.xlabel("Flow rate Q (mL/min)")
    plt.ylabel("Wall shear stress τ (Pa)")
    plt.title("Shear stress vs. flow rate")
    plt.show()
    display(Markdown(f"**Result:** τ ≈ {tau:.3f} Pa at Q = {Q_mLmin:.2f} mL/min, b = {b*1e3:.1f} mm, h = {h*1e6:.0f} µm."))

_ = interact(
    shear_stress,
    mu=FloatSlider(value=1e-3, min=0.5e-3, max=2.0e-3, step=0.1e-3, readout_format='.1e', description='μ (Pa·s)'),
    Q_mLmin=FloatSlider(value=1.0, min=0.05, max=5.0, step=0.05, description='Q (mL/min)'),
    b=FloatSlider(value=1e-2, min=2e-3, max=2e-2, step=1e-3, readout_format='.1e', description='b (m)'),
    h=FloatSlider(value=5e-4, min=1e-4, max=1e-3, step=1e-5, readout_format='.1e', description='h (m)')
)



## 3) Perfusion Design Helper — Péclet and Damköhler numbers

- **Péclet:** \(\mathrm{Pe} = \dfrac{u L}{D}\) (convection vs diffusion)  
- **Damköhler II:** \(\mathrm{Da}_{II} = \dfrac{k L^2}{D}\) (reaction vs diffusion)  
- **Damköhler I:**  \(\mathrm{Da}_{I}  = \dfrac{k L}{u}\) (reaction vs convection)

**Guidance:**  
- Low Pe (< 1): diffusion-dominated → consider increasing flow or adding channels.  
- High Pe (≫ 1): convection-dominated → check shear stress & residence time.  
- High Da: strong consumption → expect steep gradients unless perfused.


In [ ]:

def perfusion_helper(D=1e-10, k=5e-6, L=1e-3, u=100e-6):
    Pe  = u*L/D
    Da2 = k*L**2/D
    Da1 = k*L/u

    # Simple messages
    def band(val, low=0.3, high=3.0, name=""):
        if val < low:
            return f"{name}: low"
        elif val > high:
            return f"{name}: high"
        return f"{name}: moderate"

    display(Markdown(f"**Péclet (Pe)** = {Pe:.2f} &nbsp;&nbsp; "
                     f"**Damköhler II (kL²/D)** = {Da2:.2f} &nbsp;&nbsp; "
                     f"**Damköhler I (kL/u)** = {Da1:.2f}"))

    lines = [
        band(Pe, 0.3, 3.0, "Transport regime"),
        band(Da2, 0.3, 3.0, "Reaction vs diffusion"),
        band(Da1, 0.3, 3.0, "Reaction vs convection")
    ]
    display(Markdown("- " + "\n- ".join(lines)))

    # Optional simple plot: vary u and show Pe and Da1 trends
    us = np.linspace(max(1e-6, 0.2*u), 5*u, 50)
    Pe_s  = us*L/D
    Da1_s = k*L/us

    plt.figure()
    plt.plot(us*1e6, Pe_s)
    plt.xlabel("Superficial velocity u (µm/s)")
    plt.ylabel("Péclet (Pe)")
    plt.title("Effect of u on Pe")
    plt.show()

    plt.figure()
    plt.plot(us*1e6, Da1_s)
    plt.xlabel("Superficial velocity u (µm/s)")
    plt.ylabel("Damköhler I (kL/u)")
    plt.title("Effect of u on Da_I")
    plt.show()

_ = interact(
    perfusion_helper,
    D=FloatSlider(value=1e-10, min=1e-12, max=5e-10, step=1e-12, readout_format='.1e', description='D (m²/s)'),
    k=FloatSlider(value=5e-6,  min=0.0,   max=5e-5,  step=1e-7,  readout_format='.1e', description='k (1/s)'),
    L=FloatSlider(value=1e-3,  min=2e-4,  max=5e-3,  step=1e-4,  readout_format='.1e', description='L (m)'),
    u=FloatSlider(value=100e-6, min=5e-6, max=1e-3, step=5e-6, readout_format='.1e', description='u (m/s)')
)
